<table class="table">
    <tr>
        <th style="text-align:left; vertical-align: middle; width:30%">
            <img src='https://docs.google.com/uc?id=1kF5iiNhcaEXEV9FF7JqN7ZwpqSi_tf_r' style="height: 90px; ">
        </th>
        <th style="text-align:right; vertical-align: middle;">
        <font color=blue size = '4' >SMT202: Analytics Applications for Smart Living</font>
        </th>
    </tr>
</table>

## Learning Objectives

1. Perform basic data analytics on survey data
2. Use Cronbach's Alpha to evaluate internal consistency of  measurements


In [ ]:
import pandas as pd
import numpy as np

# For Alpha Cronbach 
import pingouin as pg

# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

C:\Users\jamie\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Some settings to the notebook to suppress warnings and allow columns long enough to view the data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Setting the option to print a few more columns
pd.options.display.max_colwidth = 200

### Data

In [ ]:
df = pd.read_csv('cleaned_survey.csv')
df.head()

,Q1.1,Q1.1_3_TEXT,Q1.2,Q1.3,Q1.4,Q1.4_4_TEXT,Q1.5,Q1.6,Q1.7,Q2.1,...,Q5.4_7_TEXT,Q6.1_1,Q6.1_2,Q6.1_3,Q6.1_4,Q6.1_5,Q6.2,cleaned_Q2.12,cleaned_Q4.8,cleaned_Q6.2
0,Singapore Citizen,NaN,16-19 years old,Female,Chinese,NaN,Full-time Student,Polytechnic,Singapore Polytechnic,I do not smoke at all / I am a non-smoker,...,NaN,No,Yes,Yes,Yes,Yes,"teach better coping mechanisms, educate the youth on vaping. demonising them will just make people less likely to come forward to ask for help or be transparent.",NaN,NaN,teach better coping mechanism educate youth vaping demonising make people le likely come forward ask help transparent
1,Singapore Citizen,NaN,16-19 years old,Female,Chinese,NaN,Full-time Student,Polytechnic,Singapore Polytechnic,I do not smoke at all / I am a non-smoker,...,NaN,No,No,Yes,Yes,Yes,It hasn't been studied fully yet and the liquid used for vape can be laced with drugs,NaN,NaN,studied fully yet liquid used vape laced drug
2,Singapore Citizen,NaN,16-19 years old,Female,Chinese,NaN,Full-time Student,Polytechnic,Singapore Polytechnic,I do not smoke at all / I am a non-smoker,...,NaN,No,No,Yes,Yes,Yes,Talk about it instead of acting like it is taboo.,NaN,NaN,talk instead acting like taboo
3,Permanent Resident,NaN,16-19 years old,Female,"Others, please specify",Filipino,Full-time Student,Polytechnic,Singapore Polytechnic,I do not smoke at all / I am a non-smoker,...,NaN,No,No,Yes,Yes,I don't know,I'm not sure tbh,NaN,NaN,sure tbh
4,"Others, please specify",International student,16-19 years old,Female,"Others, please specify",Korean,Full-time Student,Polytechnic,Singapore Polytechnic,I do not smoke at all / I am a non-smoker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reliability Analysis 

Visualise counts of likert scale.

In [ ]:
Q5_4_df = df[["Q5.4_1", "Q5.4_2", "Q5.4_3", "Q5.4_4", "Q5.4_5", "Q5.4_6"]].dropna()
Q5_4_1 = Q5_4_df["Q5.4_1"].value_counts()
Q5_4_2 = Q5_4_df["Q5.4_2"].value_counts()
Q5_4_3 = Q5_4_df["Q5.4_3"].value_counts()
Q5_4_4 = Q5_4_df["Q5.4_4"].value_counts()
Q5_4_5 = Q5_4_df["Q5.4_5"].value_counts()
Q5_4_6 = Q5_4_df["Q5.4_5"].value_counts()

df2 = pd.concat([Q5_4_1, Q5_4_2, Q5_4_3, Q5_4_4, Q5_4_5, Q5_4_6], axis=1)
df2 = df2.rename(columns={'Q5.4_1': 'vapesExpensive', 'Q5.4_2': 'moreHarmful', 'Q5.4_3': 'smellBetter',
                          'Q5.4_4': 'getAddicted', 'Q5.4_5': 'secondHandSmoke', 'Q5.4_6': 'dkWhereToGet'})
df2.fillna(0).astype(int)

,vapesExpensive,moreHarmful,smellBetter,getAddicted,secondHandSmoke,secondHandSmoke
Strongly Agree,20,29,8,37,39,39
Agree,11,13,14,5,3,3
Strongly Disagree,8,1,14,1,1,1
Disagree,4,0,7,0,0,0


#### Choose subscale (category of questions)

Let's assume we want to analyse the internal consistency of the set of questions captured in the varaiables in Q5.4
- Q5.4_1 - It is expensive
- Q5.4_2 - It is more harmful than cigarettes
- Q5.4_3 - Electronic nicotine delivery systems (ENDS)/e-cigarettes/vapes tastes and smells better than cigarettes
- Q5.4_4 - It is harmful to health
- Q5.4_5 - It may be addictive
- Q5.4_6 - I don’t know where to get it
- Q5.4_7 - Others, please specify (please enter NA and select "Agree" if there are no others)

Q5.4_7 is excluded since only 2 of the 47 people who answered this question stated an answer for "Others"

In [ ]:
# Dataframe that contain the variables
Q5_4_df.head()

,Q5.4_1,Q5.4_2,Q5.4_3,Q5.4_4,Q5.4_5,Q5.4_6
1,Agree,Strongly Agree,Agree,Agree,Strongly Agree,Disagree
3,Agree,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Disagree
6,Strongly Disagree,Agree,Disagree,Strongly Agree,Strongly Agree,Strongly Disagree
7,Agree,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Disagree
8,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree


#### Mapping the categorical content into numbers

In [ ]:
scale_mapper = {"Strongly Disagree":1, "Disagree":2, "Agree":3, "Strongly Agree":4}

Q5_4_df["Q5.4_1"].replace(scale_mapper, inplace=True)
Q5_4_df["Q5.4_2"].replace(scale_mapper, inplace=True)
Q5_4_df["Q5.4_3"].replace(scale_mapper, inplace=True)
Q5_4_df["Q5.4_4"].replace(scale_mapper, inplace=True)
Q5_4_df["Q5.4_5"].replace(scale_mapper, inplace=True)
Q5_4_df["Q5.4_6"].replace(scale_mapper, inplace=True)

Q5_4_df.head()

,Q5.4_1,Q5.4_2,Q5.4_3,Q5.4_4,Q5.4_5,Q5.4_6
1,3,4,3,3,4,2
3,3,4,3,4,4,2
6,1,3,2,4,4,1
7,3,3,3,4,4,1
8,4,4,4,4,4,4


In [ ]:
pg.cronbach_alpha(data=Q5_4_df)

(0.6373774509803921, array([0.439, 0.783]))

This shows that the internal consistency is questionable. 


### Addressing Inconsistency


#### 1. Dropping One Question

In [ ]:
df3 = Q5_4_df.drop(columns=['Q5.4_3'])

In [ ]:
df3.head()

,Q5.4_1,Q5.4_2,Q5.4_4,Q5.4_5,Q5.4_6
1,3,4,3,4,2
3,3,4,4,4,2
6,1,3,4,4,1
7,3,3,4,4,1
8,4,4,4,4,4


In [ ]:
pg.cronbach_alpha(data=df3)

(0.7307063242064454, array([0.578, 0.84 ]))

Cronbach's Alpha realiability score increased to 0.731 from 0.637 after Q5.4_3 was dropped.

Cronbach's Alpha when each question dropped:
- Q5.4_1 dropped = 0.508 (became worse)
- Q5.4_2 dropped = 0.547 (became worse)
- Q5.4_4 dropped = 0.565 (became worse)
- Q5.4_5 dropped = 0.564 (became worse)
- Q5.4_6 dropped = 0.625 (became worse)

#### 2. Reverse Coding


In [ ]:
# Create a mapper which is reverse coding
scale_mapper2 = {"Strongly Disagree":4, "Disagree":3, "Agree":2, "Strongly Agree":1}

df4 = Q5_4_df

df4["Q5.4_1"].replace(scale_mapper, inplace=True)
df4["Q5.4_2"].replace(scale_mapper, inplace=True)
df4["Q5.4_3"].replace(scale_mapper2, inplace=True)
df4["Q5.4_4"].replace(scale_mapper, inplace=True)
df4["Q5.4_5"].replace(scale_mapper, inplace=True)
df4["Q5.4_6"].replace(scale_mapper, inplace=True)

df4.head()

,Q5.4_1,Q5.4_2,Q5.4_3,Q5.4_4,Q5.4_5,Q5.4_6
1,3,4,3,3,4,2
3,3,4,3,4,4,2
6,1,3,2,4,4,1
7,3,3,3,4,4,1
8,4,4,4,4,4,4


In [ ]:
pg.cronbach_alpha(data=df4)

(0.6373774509803921, array([0.439, 0.783]))

Dropping the entire question Q5.4_3 achieve better reliability. However, reverse coding makes more sense since Q5.4_3 (Electronic nicotine delivery systems (ENDS)/e-cigarettes/vapes tastes and smells better than cigarettes) is the only positive question out of the 6 questions. The other five are all asked in a negative light.

Thus, it is better to reverse code Q5.4_3 than to drop the whole question.